Referencias (para equipo docente):
- bounding box chile continental: extraído manualmente de [boundingbox.klokantech.com](https://boundingbox.klokantech.com/) \[[[-79.6157454929,-56.72500008],[-66.07534742,-56.72500008],[-66.07534742,-17.49839982],[-79.6157454929,-17.49839982],[-79.6157454929,-56.72500008]]\]  
**lon**: \[-79.6157454929, -66.07534742\], **lat**\[-56.72500008, -17.49839982\]
- extender singledispatch a métodos de [stackoverflow](https://stackoverflow.com/questions/24601722/how-can-i-use-functools-singledispatch-with-instance-methods)

In [ ]:
%load_ext autoreload
%autoreload 2
import requests
import json
import numpy as np
from scipy import stats
from datetime import date, timedelta
from time import sleep

_An Application Programming Interface (API) is a set of routines, protocols, and tools for building software applications. Basically, an API specifies how software components should interact._ - [By Vangie Beal in Webopedia](https://www.webopedia.com/TERM/A/API.html)

In this workshop we will work with an special types of API's, which are the [Web API's](https://en.wikipedia.org/wiki/Web_API) and particularly with Web API's which are designed to download data from servers in a programmatic way.

# Ejercicio 1
**Objetivo**: Calcular estadísticas descriptivas sobre los movimientos telúricos ocurridos en Chile en los últimos 100 años, haciendo uso de algunas de las herramientas del curso.

En pos de dicho objetivo utilizaremos una [API](https://earthquake.usgs.gov/fdsnws/event/1/) gratuita del _United States Geological Survey_ (USGS). Una API - Aplication Programming Interface o interfaz de programación de aplicaciones - es un conjunto de rutinas, protocolos y herramientas para crear aplicaciones de software. Básicamente, una API especifica cómo deben interactuar los componentes de software. - [Por Vangie Beal en Webopedia](https://www.webopedia.com/TERM/A/API.html)

En este ejercicio trabajaremos con un tipo especial de API, que son las [web API](https://en.wikipedia.org/wiki/Web_API) y particularmente con las web API que están diseñadas para descargar datos de servidores en un forma programática

Los pasos a seguir serán los siguientes:
1. Construir las consultas
2. Recolectar los datos
3. Analizar los resultados


## Preliminares

Antes de comenzar, preste especial atención a al siguiente decorador. Este extiende el decorador `functools.singledispatch` para manejar métodos en vez de funciones. Normalmente, este decorador está disponible a partir de python 3.8 [link](https://docs.python.org/3/library/functools.html#functools.singledispatchmethod), pero como es posible observar a continuación, con la ayuda de 
las herramientas que esta librería ya proporciona en Python 3.7, es posible extender fácilmente la funcionalidad de `functools.singledispatch` como es deseado:


In [ ]:
from functools import singledispatch, update_wrapper

def singledispatchmeth(func):
    
    despachador = singledispatch(func)
    def wrapper(*args, **kw):
        return despachador.dispatch(args[1].__class__)(*args, **kw)
    wrapper.register = despachador.register
    update_wrapper(wrapper, func)
    return wrapper

**OBS**: Este decorador será utilizado en dos ocasiones en el ejercicio, por lo que vale la pena comprender su funcionamiento.

## Construcción de consultas

Como está explicado en la documentación de la [API](https://earthquake.usgs.gov/fdsnws/event/1/) para hacer una consulta podemos definir una diversidad de parámetros. Los que serán útiles para el ejercicio son:
- Geográficos: que servirán para definir la zona geográfica de interés. Estos son: `minlatitude`, `minlongitude`, `maxlatitude` y `maxlongitude`.
- Temporales: servirán para definir el periodo de consulta. Se trata de: `starttime` y `endtime`.

Dado que la API entrega un máximo de 20.000 respuestas por consulta y que Chile es un país con mucha actividad sísmica es necesario realizar múltiples consultas para capturar todos movimientos telúricos en cuestión. Las multiples consultas variarán su periodo de interés, pero la zona geográfica queadrá fija .


Ya que es requerido trabajar con variables temporales, se hace uso de la clase `date` de la librería `datetime`. Para denotar el primero de enero de 1920, el objeto se puede inicializar con una tupla de largo 3 que contiene sólo enteros que serán interpretados de la forma `(año, mes, día)`. Es decir:

In [ ]:
primera_fecha = date(1920, 1, 1)
print(f'La representacion del objeto `primera_fecha` es: {repr(primera_fecha)}')
print(f'El string asociado `primera_fecha` es: {primera_fecha}')

1. Extienda la clase `date` de la librería `datetime` a una nueva clase que denominaremos `DateConSuma`, en la cual el operador `+` se comporte de igual forma que la clase `date`, excepto al operar con una tupla de largo tres que contenga sólo enteros. En este caso, la clase debe interpretar la tupla de la forma `(años a sumar, meses a sumar, días a sumar)`.
    - Utilice el decorador definido arriba `singledispatchmeth`.
    - Recuerde hacer uso de control de flujo para asegurar que se está operando con una tupla de las caracrerísticas  especificadas.
    - Haga uso de la función `super` adecuadamente. 

In [ ]:
from datetime import date

class DateConSuma(date):
    
    @singledispatchmeth
    def __add__(self, sumando):
        return super().__add__(sumando)
    
    @__add__.register(tuple)
    def _(self, sumando):
        assert len(sumando) == 3, 'La operación `+` esta solo definida para tuplas de largo 3'
        assert all([isinstance(x, int) for x in sumando])
        nueva_fecha = self.replace(year=self.year + sumando[0])
        nueva_fecha = nueva_fecha.replace(month=self.month + sumando[1])
        nueva_fecha = nueva_fecha.replace(day=self.day + sumando[2])
        return nueva_fecha

- Compruebe que el resultado es el deseado:

In [ ]:
fecha_con_suma = DateConSuma(1920, 1, 1) 
fecha_con_suma + (10, 0, 0)

Note además que las operaciones de comparación funcionan como lo esperado:

In [ ]:
f'El dia de hoy {date.today()} sucedio antes que {fecha_con_suma}? {fecha_con_suma >= date.today()}'

Ahora que ya se tienen herramientas para trabajar con fechas, crearemos un iterable que entregue tuplas con las fechas necesarias para realizar las consultas.

2. Defina la clase `IteratorFechas` que entregue objetos de la clase `DateConSuma` espaciados cada 10 años comenzando en 1920-01-01 y finalizando en 2020-01-01. Para ello:
    - Defina los atributos públicos `desde_fecha` y `dia_presente` en el constructor, ambos de la clase `DateConSuma`
    - Defina el método mágico `__iter__` donde simplemente retorne el objeto.
    - Defina el método mágico `__next__` donde retorne una tupla de largo 2 que retorne los valores espaciados por 5 años. El output de las 3 primeras y últimas iteraciones debería retornar las siguientes tuplas:
        ```python
        ('DateConSuma(1920, 1, 1)', 'DateConSuma(1925, 1, 1)')
        ('DateConSuma(1925, 1, 1)', 'DateConSuma(1930, 1, 1)')
        ('DateConSuma(1930, 1, 1)', 'DateConSuma(1935, 1, 1)')
        [...]
        ('DateConSuma(2005, 1, 1)', 'DateConSuma(2010, 1, 1)')
        ('DateConSuma(2010, 1, 1)', 'DateConSuma(2015, 1, 1)')
        ('DateConSuma(2015, 1, 1)', 'DateConSuma(2020, 1, 1)')

        ```
      Una vez entregada la última tupla i.e. ('DateConSuma(2010, 1, 1)', 'DateConSuma(2020, 1, 1)') es necesario que `__next__` levante la excepción `StopIteration` la próxima vez que sea llamado.

_Hint_ : Es necesario recordar como funciona un **iterable**. Además puede ser útil un método ya utilizado de la clase `date`.

    equipo docente: El siguiente también puede ser escrito como un generador.

In [ ]:
class IteratorFechas():
    
    def __init__(self):
        self.desde_fecha = DateConSuma(1920, 1, 1)
        self.dia_presente = date.today()
    
    def __iter__(self):
        return self
    
    def __next__(self):
        hasta_fecha = self.desde_fecha + (5, 0, 0)
        
        if hasta_fecha <= self.dia_presente:
            viejo_desde_fecha = self.desde_fecha
            self.desde_fecha = hasta_fecha
            return (viejo_desde_fecha, hasta_fecha)
        
        else:
            raise StopIteration()

- Compruebe que la clase `GeneradorFechas` funciona como es esperado.

In [ ]:
for desde_, hasta_ in IteratorFechas():
    print((repr(desde_), repr(hasta_)))

\todo: explicar como se generan las consultas y proponer parte de la clase como ejercicio
3. Cree la clase `GeneradorConsulta`.

In [ ]:
class GeneradorConsulta(IteratorFechas):
        
    def __init__(self):
        
        super().__init__()
        # atributos fijos de consulta
        self.params_dict = {
            'minlatitude': -56.72500008, 
            'minlongitude': -80,
            'maxlatitude': -17.49839982,
            'maxlongitude': -66.07534742,
        }
    
    # privada que no puede ser modificada por el usuario
    @property
    def url_api(self):
        self.__url_api = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
        return self.__url_api

    def __iter__(self):
        return self

    def __next__(self):
        
        desde_fecha, hasta_fecha = super().__next__()
        iter_dict = {
            'starttime': desde_fecha,
            'endtime': hasta_fecha
        }
        iter_dict = {**iter_dict, **self.params_dict}
        lista_parametros = ['&'+str(k)+'='+str(v) for k, v in iter_dict.items()]
        return self.url_api + ''.join(lista_parametros)



- Compruebe que el resultado es el deseado:

In [ ]:
for consulta_str in GeneradorConsulta():
    print(consulta_str)

## Recolección de datos

4. Extienda dict

In [ ]:
class DictDeListas(dict):
    
    def extiende_listas(self, dict_de_listas):
        
        for k, l in dict_de_listas.items():
            
            self[k] += l
            

- Compruebe que el resultado es el deseado:

In [ ]:
primer_dict_de_listas = DictDeListas({
    'id': ['iscgem908354', 'iscgem908107','iscgem908989']
})
primer_dict_de_listas.extiende_listas({
    'id': ['iscgem908986', 'iscgem908971', 'iscgem908457']
})
primer_dict_de_listas

5. Recolecte datos segun `funciones_dict`

In [ ]:
funciones_dict = {
    'id': lambda x: x['id'],
    'lugar': lambda x: x['properties']['place'],
    'timestamp': lambda x: x['properties']['time'],
    'magnitud': lambda x: x['properties']['mag'],
    'longitud': lambda x: x['geometry']['coordinates'][0],
    'latitud': lambda x: x['geometry']['coordinates'][1]
}

In [ ]:
observaciones_dict = DictDeListas()
for consulta_str in GeneradorConsulta():
    
    with requests.get(consulta_str) as respuesta:
        
        assert respuesta.ok, f'Respuesta erronea de codigo: {respuesta.status_code}'
        
        respuesta_dict = respuesta.json()
        respuesta_features_lista = respuesta_dict['features']
        
        # reporta numero de resultados
        print(f'{len(respuesta_dict["features"])} observaciones en consulta {consulta_str}' )
        
        propiedades_iter = [dict(filter(
            lambda x: x[0] in ['place', 'time', 'mag'], obs['properties'].items()))\
            for obs in respuesta_features_lista]
        
        iter_dict = DictDeListas()
        
        for k, func in funciones_dict.items():
            
            iter_dict[k] = list(map(func, respuesta_features_lista))
            
        if observaciones_dict:
            observaciones_dict.extiende_listas(iter_dict)
            
        else:
            observaciones_dict = DictDeListas(iter_dict.copy())
        
        # espera 5 segundos antes de realizar la próxima consulta
        sleep(5)
        
        

## Analisis de datos

6. Transforme las listas en numpy array segun 

In [ ]:
dtype_dict = {
    'id': np.dtype('O'),
    'lugar': np.dtype('O'),
    'timestamp': np.dtype(float),
    'magnitud': np.dtype(float),
    'longitud': np.dtype(float),
    'latitud': np.dtype(float)
}

In [ ]:
observaciones_arr_dict = {k: np.array(v, dtype=dtype_dict[k]) for k, v in observaciones_dict.items()}

7. Adapte el código de singledispatchmeth 

In [ ]:
def numpydispatch(func):
    dispatcher = singledispatch(func)
    def wrapper(*args, **kw):
        return dispatcher.dispatch(args[0][0].__class__)(*args, **kw)
    wrapper.register = dispatcher.register
    update_wrapper(wrapper, func)
    return wrapper

8. Complete el siguiente código

In [ ]:
@numpydispatch
def muestra_estadisticas_descriptivas(array):
    lista_str = []
    lista_str.append(f"numero de observaciones: {len(array):,}")
    print('\n'.join(lista_str))
    
@muestra_estadisticas_descriptivas.register(np.float64)
def _(array):
    lista_str = []
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(f"media: {np.nanmean(array):.2f}")
    lista_str.append(f"mediana: {np.nanmedian(array):.2f}")
    lista_str.append(f"primer cuartil: {np.nanquantile(array, .25):.2f}")
    lista_str.append(f"tercer cuartil: {np.nanquantile(array, .75):.2f}")
    lista_str.append(f"minimo: {np.nanmin(array)}")
    lista_str.append(f"maximo: {np.nanmax(array)}")
    print('\n'.join(lista_str))
        
@muestra_estadisticas_descriptivas.register(str)
def _(array):
    
    resultados_moda = stats.mode(array, nan_policy="omit")
    
    lista_str = []
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(f'moda: {resultados_moda.mode[0]} ({resultados_moda.count[0]:,} ocurrencias)')
    print('\n'.join(lista_str))
        

# @muestra_estadisticas_descriptivas.register()
    

- Compruebe que el resultado es el deseado:

In [ ]:
# test_dict = {}
# test_dict['magnitud'] = np.array(observaciones_dict['magnitud'], dtype=float)
# test_dict['lugar'] = np.array(observaciones_dict['lugar'], dtype=np.object)

for k, val in observaciones_arr_dict.items():
    
    if k == 'id': 
        continue
    
    print(f'\nVariable: {k}')
    muestra_estadisticas_descriptivas(val)

9. Filtrar filas que no estén en Chile  


    \pendiente!